**Testing the KD Tree Balance under different splttings**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Generate synthetic data

def generate_data(n_samples=200, n_dims=2, distribution='uniform'):
    if distribution == 'uniform':
        return np.random.uniform(-1, 1, size=(n_samples, n_dims))
    elif distribution == 'gaussian':
        return np.random.normal(0, 0.5, size=(n_samples, n_dims))
    elif distribution == 'clusters':
        num_clusters = min(4, n_samples // 50)  # Set number of clusters dynamically
        centers = np.random.uniform(-1, 1, size=(num_clusters, n_dims))
        data = np.vstack([np.random.normal(loc=center, scale=0.2, size=(n_samples // num_clusters, n_dims)) for center in centers])
        return data

# Sliding Midpoint of Widest Side heuristic
def sliding_midpoint_split(data):
    ranges = np.ptp(data, axis=0)  # Peak-to-peak (max - min) per dimension
    split_dim = np.argmax(ranges)  # Dimension with the widest spread
    midpoint = (np.max(data[:, split_dim]) + np.min(data[:, split_dim])) / 2

    # Ensure a balanced split (slide if necessary)
    left = data[data[:, split_dim] <= midpoint]
    right = data[data[:, split_dim] > midpoint]

    if min(len(left), len(right)) < len(data) * 0.1:  # Too unbalanced, adjust
        median = np.median(data[:, split_dim])
        left = data[data[:, split_dim] <= median]
        right = data[data[:, split_dim] > median]

    return split_dim, left, right

# Maximum Variance heuristic
def max_variance_split(data):
    variances = np.var(data, axis=0)
    split_dim = np.argmax(variances)  # Dimension with max variance
    median = np.median(data[:, split_dim])

    left = data[data[:, split_dim] <= median]
    right = data[data[:, split_dim] > median]

    return split_dim, left, right

# Round-Robin Median Split heuristic
def round_robin_split(data, current_dim):
    split_dim = current_dim % data.shape[1]  # Cycle through dimensions
    median = np.median(data[:, split_dim])

    left = data[data[:, split_dim] <= median]
    right = data[data[:, split_dim] > median]

    return split_dim, left, right

# Compare splitting methods
def evaluate_splitting(data, method):
    split_dim, left, right = method(data)
    balance_ratio = min(len(left), len(right)) / max(len(left), len(right))
    return split_dim, balance_ratio

# Compare round robin with previous methods
def evaluate_round_robin_splitting(data, dim_counter):
    split_dim, left, right = round_robin_split(data, dim_counter)
    balance_ratio = min(len(left), len(right)) / max(len(left), len(right))
    return split_dim, balance_ratio

# Run experiments on different datasets
distributions = ['uniform', 'gaussian', 'clusters']
results = []
dim_counter = 0
n_samples = 1_000
n_dims = 6  # Now supports arbitrary dimensions

for dist in distributions:
    data = generate_data(n_samples=n_samples, n_dims=n_dims, distribution=dist)
    split_dim_smw, balance_smw = evaluate_splitting(data, sliding_midpoint_split)
    split_dim_var, balance_var = evaluate_splitting(data, max_variance_split)
    split_dim_rr, balance_rr = evaluate_round_robin_splitting(data, dim_counter)
    dim_counter += 1  # Increment for round robin dimension selection

    results.append([dist, split_dim_smw, balance_smw, split_dim_var, balance_var, split_dim_rr, balance_rr])

# Convert results into a DataFrame and display
df_results = pd.DataFrame(results, columns=[
    'Distribution', 'SMW Split Dim', 'SMW Balance', 'Var Split Dim', 'Var Balance', 'RR Split Dim', 'RR Balance'
])

# Display the dataframe in Google Colab
display(df_results)


Distribution  SMW Split Dim  SMW Balance  Var Split Dim  Var Balance  \
0      uniform              3     0.980198              4          1.0   
1     gaussian              3     0.677852              4          1.0   
2     clusters              5     1.000000              5          1.0   

   RR Split Dim  RR Balance  
0             0         1.0  
1             1         1.0  
2             2         1.0

**KD Tree Test Leaf Size vs Non-Leaf Size**

In [ ]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math


In [ ]:
import random
import numpy as np
from numpy import ndarray

In [ ]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 7 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [ ]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/higgs_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [ ]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, LQ)
]

"""
test_combinations = [
    (SB, MQ),
    (LB, SQ),
    (MB, MQ),
    (MB, LQ),
    (LB, MQ),
    (MB, SQ),
    (SB, SQ),
    (LB, LQ),
    (SB, LQ),
    (MB, MQ)
]
""";

In [ ]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**QuickSelect Algorithm**

In [ ]:
from collections import deque
from typing import List, Tuple

def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot



def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key

**Original KD Tree**

In [ ]:
import numpy as np
from collections import deque
from typing import List, Tuple, Deque

class KDNode:
    def __init__(self, point=None, left=None, right=None, parent=None):
        self.point = point
        self.split_dim = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.equal_points = []  # Store points with equal coordinates

class KDTree:
    def __init__(self):
        self.points = None
        self.root = None
        self.dim = None

    def find_best_split_dimension(self, points: List, start_dim: int) -> Tuple[int, bool]:
        """Find the best dimension to split on, starting from start_dim.
        Returns (split_dimension, all_equal_flag)"""
        if len(points) == 0:
            return 0, True

        for d in range(self.dim):
            # Try dimensions in order starting from start_dim
            curr_dim = (start_dim + d) % self.dim
            coords = [p[curr_dim] for p in points]
            if not all(x == coords[0] for x in coords[1:]):
                return curr_dim, False
        return 0, True  # All dimensions have equal coordinates

    def build(self, points=None):
        """Build a KD tree using the collection of points."""
        if points is None or len(points) == 0:
            return None

        self.points = points
        self.dim = len(points[0])

        # Create root node first
        self.root = KDNode()

        # Stack will store (node, points, depth)
        stack = deque([(self.root, points, 0)])

        while stack:
            node, points, depth = stack.pop()

            if len(points) == 0:
                continue

            # Find best splitting dimension
            split_dim, all_equal = self.find_best_split_dimension(points, depth % self.dim)

            if all_equal:
                # All points are identical across all dimensions
                node.leaf = True
                node.point = points[0]
                node.equal_points = points[1:]  # Store other equal points
                continue

            # Set splitting dimension
            node.split_dim = split_dim

            # Get coordinates for the splitting dimension
            coords = [(p[split_dim], i) for i, p in enumerate(points)]

            # Find median using quickselect
            k = len(points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            median_point = points[med_[1]]

            # Set node properties
            node.point = median_point

            # Split points, handling equal values correctly
            left_points = []
            right_points = []
            equal_points = []

            for p in points:
                if p[split_dim] < median_val:
                    left_points.append(p)
                elif p[split_dim] > median_val:
                    right_points.append(p)
                elif not np.array_equal(p, median_point):  # Equal to median but not the median point itself
                    equal_points.append(p)

            # Store points equal to median
            node.equal_points = equal_points

            # Create and add children
            if left_points:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if right_points:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))

            if not left_points and not right_points:
                node.leaf = True

        return self.root

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def search_(self, root: KDNode = None, query: List = None) -> Tuple:
        """Find nearest neighbor using a stack-based approach with L2 distance."""
        if root is None:
            return None, None

        d_best = float('inf')
        d_best_pt = None

        # Stack will store (node, bound) pairs
        stack = deque([(root, 0)])

        while stack:
            node, bound = stack.pop()

            if node is None or bound >= d_best:
                continue

            # Check current point
            curr_dist = self.distance(node.point, query)
            if curr_dist < d_best:
                d_best = curr_dist
                d_best_pt = node.point

            # Check equal points stored in the node
            for p in node.equal_points:
                curr_dist = self.distance(p, query)
                if curr_dist < d_best:
                    d_best = curr_dist
                    d_best_pt = p

            if node.leaf:
                continue

            # Calculate distance to splitting plane
            split_dim = node.split_dim
            diff = query[split_dim] - node.point[split_dim]
            dist_to_plane = abs(diff)

            # Push subtrees onto stack (further subtree first so closer one is popped first)
            if diff <= 0:
                # Query is in left subtree
                if node.right_tr:
                    # Only add if bound could be better than current best
                    if dist_to_plane < d_best:
                        stack.append((node.right_tr, dist_to_plane))
                if node.left_tr:
                    stack.append((node.left_tr, bound))
            else:
                # Query is in right subtree
                if node.left_tr:
                    if dist_to_plane < d_best:
                        stack.append((node.left_tr, dist_to_plane))
                if node.right_tr:
                    stack.append((node.right_tr, bound))

        return d_best, d_best_pt

In [ ]:
# Test code KD  Tree


results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    kd_tree = KDTree()
    kd_tree.build(points=current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = kd_tree.search_(root=kd_tree.root, query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of KD Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of KD Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 3941.94it/s]


Build Time: 0.26 sec, Search Time: 0.03 sec

2: === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 48.91it/s]


Build Time: 46.67 sec, Search Time: 0.41 sec

3: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 164.62 sec, Search Time: 32.29 sec

9: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:32<00:00, 15.25it/s]


Build Time: 172.65 sec, Search Time: 32.61 sec

10: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:07<00:00, 14.02it/s]

Build Time: 175.76 sec, Search Time: 7.10 sec

=== Summary of KD Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 0.26 sec, Search: 0.03 sec
Data: 1000000 points, Queries:   20 => Build: 46.67 sec, Search: 0.41 sec
Data:  100000 points, Queries:  100 => Build: 50.73 sec, Search: 2.38 sec
Data:  100000 points, Queries:  500 => Build: 57.08 sec, Search: 11.66 sec
Data: 1000000 points, Queries:  100 => Build: 107.04 sec, Search: 4.75 sec
Data:  100000 points, Queries:   20 => Build: 116.00 sec, Search: 0.96 sec
Data:   10000 points, Queries:   20 => Build: 115.47 sec, Search: 0.78 sec
Data: 1000000 points, Queries:  500 => Build: 164.62 sec, Search: 32.29 sec
Data:   10000 points, Queries:  500 => Build: 172.65 sec, Search: 32.61 sec
Data:  100000 points, Queries:  100 => Build: 175.76 sec, Search: 7.10 sec
--------------------------------------------

=== Summary of KD Tree Tests ===

Total Build Time: 1006.29 sec, Total Search Time: 92.97 sec


**KD Tree Node Size**

In [ ]:
import numpy as np
from collections import deque
from typing import List, Tuple, Deque

class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Store multiple points in a node
        self.split_dim = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent

class KDTree:
    def __init__(self, bucket_size=40):
        self.points = None
        self.root = None
        self.dim = None
        self.bucket_size = bucket_size  # Maximum number of points per node

    def find_best_split_dimension(self, points: List, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            coords = [p[curr_dim] for p in points]
            if not all(x == coords[0] for x in coords[1:]):
                return curr_dim, False
        return 0, True  # All dimensions have equal coordinates

    def build(self, points=None):
        if points is None or len(points) == 0:
            return None
        self.points = points
        self.dim = len(points[0])
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, points, depth = stack.pop()

            if len(points) <= self.bucket_size:
                node.leaf = True
                node.points = points  # Store all points in the leaf node
                continue

            split_dim, all_equal = self.find_best_split_dimension(points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = points  # Store all points in the leaf node
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(points)]
            k = len(points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            median_point = points[med_[1]]

            left_points = []
            right_points = []
            equal_points = []
            for p in points:
                if p[split_dim] < median_val:
                    left_points.append(p)
                elif p[split_dim] > median_val:
                    right_points.append(p)
                else:
                    equal_points.append(p)

            node.points = equal_points
            if left_points:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if right_points:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if not left_points and not right_points:
                node.leaf = True

        return self.root

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def search_(self, root: KDNode = None, query: List = None) -> Tuple:
        if root is None:
            return None, None
        d_best = float('inf')
        d_best_pt = None
        stack = deque([(root, 0)])

        while stack:
            node, bound = stack.pop()
            if node is None or bound >= d_best:
                continue

            for p in node.points:  # Check all points in the node
                curr_dist = self.distance(p, query)
                if curr_dist < d_best:
                    d_best = curr_dist
                    d_best_pt = p

            if node.leaf:
                continue

            split_dim = node.split_dim
            diff = query[split_dim] - node.points[0][split_dim]
            dist_to_plane = abs(diff)

            if diff <= 0:
                if node.right_tr and dist_to_plane < d_best:
                    stack.append((node.right_tr, dist_to_plane))
                if node.left_tr:
                    stack.append((node.left_tr, bound))
            else:
                if node.left_tr and dist_to_plane < d_best:
                    stack.append((node.left_tr, dist_to_plane))
                if node.right_tr:
                    stack.append((node.right_tr, bound))

        return d_best, d_best_pt


In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
node_sz = 40

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(bucket_size=node_sz)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = kdtree.search_(kdtree.root, query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of KD Tree Node Size Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of KD Tree Node Size Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 2904.18it/s]

Build Time: 0.12 sec, Search Time: 0.03 sec

2: === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===



Querying: 100%|██████████| 20/20 [00:00<00:00, 47.33it/s]


Build Time: 31.43 sec, Search Time: 0.42 sec

3: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 127.95 sec, Search Time: 31.85 sec

9: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:32<00:00, 15.31it/s]


Build Time: 132.71 sec, Search Time: 32.49 sec

10: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:07<00:00, 13.91it/s]

Build Time: 138.91 sec, Search Time: 7.15 sec

=== Summary of KD Tree Node Size Tests ===
Data:   10000 points, Queries:  100 => Build: 0.12 sec, Search: 0.03 sec
Data: 1000000 points, Queries:   20 => Build: 31.43 sec, Search: 0.42 sec
Data:  100000 points, Queries:  100 => Build: 35.81 sec, Search: 2.42 sec
Data:  100000 points, Queries:  500 => Build: 39.96 sec, Search: 11.91 sec
Data: 1000000 points, Queries:  100 => Build: 79.89 sec, Search: 4.85 sec
Data:  100000 points, Queries:   20 => Build: 87.28 sec, Search: 0.98 sec
Data:   10000 points, Queries:   20 => Build: 87.86 sec, Search: 0.71 sec
Data: 1000000 points, Queries:  500 => Build: 127.95 sec, Search: 31.85 sec
Data:   10000 points, Queries:  500 => Build: 132.71 sec, Search: 32.49 sec
Data:  100000 points, Queries:  100 => Build: 138.91 sec, Search: 7.15 sec
--------------------------------------------

=== Summary of KD Tree Node Size Tests ===

Total Build Time: 761.92 sec, Total Search Time: 92.82 sec


**KD Tree Node Size, PQ Search**

In [ ]:
import numpy as np
import random
import heapq
from collections import deque
from numpy import ndarray
from typing import List, Tuple, Deque

class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Store multiple points in a node
        self.split_dim = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent

class KDTree:
    def __init__(self, bucket_size=40):
        self.points = None
        self.root = None
        self.dim = None
        self.bucket_size = bucket_size  # Maximum number of points per node

    def find_best_split_dimension(self, points: List, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            coords = [p[curr_dim] for p in points]
            if not all(x == coords[0] for x in coords[1:]):
                return curr_dim, False
        return 0, True  # All dimensions have equal coordinates

    def build(self, points=None):
        if points is None or len(points) == 0:
            return None
        self.points = points
        self.dim = len(points[0])
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, points, depth = stack.pop()

            if len(points) <= self.bucket_size:
                node.leaf = True
                node.points = points  # Store all points in the leaf node
                continue

            split_dim, all_equal = self.find_best_split_dimension(points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = points  # Store all points in the leaf node
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(points)]
            k = len(points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            median_point = points[med_[1]]

            left_points = []
            right_points = []
            equal_points = []
            for p in points:
                if p[split_dim] < median_val:
                    left_points.append(p)
                elif p[split_dim] > median_val:
                    right_points.append(p)
                else:
                    equal_points.append(p)

            node.points = equal_points
            if left_points:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if right_points:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if not left_points and not right_points:
                node.leaf = True

        return self.root

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def search_(self, root: KDNode = None, query: List = None) -> Tuple:
        if root is None:
            return None, None
        d_best = float('inf')
        d_best_pt = None
        priority_queue = []  # Min-heap for best-first search
        heapq.heappush(priority_queue, (0, id(root), root))  # Store node ID to ensure uniqueness

        while priority_queue:
            bound, _, node = heapq.heappop(priority_queue)
            if node is None or bound >= d_best:
                continue

            for p in node.points:  # Check all points in the node
                curr_dist = self.distance(p, query)
                if curr_dist < d_best:
                    d_best = curr_dist
                    d_best_pt = p

            if node.leaf:
                continue

            split_dim = node.split_dim
            diff = query[split_dim] - node.points[0][split_dim]
            dist_to_plane = abs(diff)

            # Always push both children, closer one first
            if diff <= 0:
                if node.left_tr:
                    heapq.heappush(priority_queue, (0, id(node.left_tr), node.left_tr))
                if node.right_tr and dist_to_plane < d_best:
                    heapq.heappush(priority_queue, (dist_to_plane, id(node.right_tr), node.right_tr))
            else:
                if node.right_tr:
                    heapq.heappush(priority_queue, (0, id(node.right_tr), node.right_tr))
                if node.left_tr and dist_to_plane < d_best:
                    heapq.heappush(priority_queue, (dist_to_plane, id(node.left_tr), node.left_tr))

        return d_best, d_best_pt


In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
node_sz = 40

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(bucket_size=node_sz)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = kdtree.search_(kdtree.root, query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of KD Tree Node Size, PQ Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of KD Tree Node Size, PQ Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 2681.23it/s]

Build Time: 0.17 sec, Search Time: 0.04 sec

2: === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===



Querying: 100%|██████████| 20/20 [00:00<00:00, 47.61it/s]


Build Time: 32.99 sec, Search Time: 0.42 sec

3: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 120.20 sec, Search Time: 32.04 sec

9: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:32<00:00, 15.22it/s]


Build Time: 126.85 sec, Search Time: 32.67 sec

10: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:07<00:00, 13.66it/s]

Build Time: 133.57 sec, Search Time: 7.28 sec

=== Summary of KD Tree Node Size, PQ Search Tests ===
Data:   10000 points, Queries:  100 => Build: 0.17 sec, Search: 0.04 sec
Data: 1000000 points, Queries:   20 => Build: 32.99 sec, Search: 0.42 sec
Data:  100000 points, Queries:  100 => Build: 37.26 sec, Search: 2.43 sec
Data:  100000 points, Queries:  500 => Build: 40.58 sec, Search: 11.97 sec
Data: 1000000 points, Queries:  100 => Build: 79.62 sec, Search: 4.87 sec
Data:  100000 points, Queries:   20 => Build: 83.66 sec, Search: 0.97 sec
Data:   10000 points, Queries:   20 => Build: 85.75 sec, Search: 0.72 sec
Data: 1000000 points, Queries:  500 => Build: 120.20 sec, Search: 32.04 sec
Data:   10000 points, Queries:  500 => Build: 126.85 sec, Search: 32.67 sec
Data:  100000 points, Queries:  100 => Build: 133.57 sec, Search: 7.28 sec
--------------------------------------------

=== Summary of KD Tree Node Size, PQ Search Tests ===

Total Build Time: 740.66 sec, Total Search Time: 93.4

**VP Tree Construction Methods First Point vs Max Var**

In [ ]:
import numpy as np
from collections import deque
import numpy as np

class VPNode:
    def __init__(self, point=None):
        self.point = point  # Vantage point
        self.mu = 0  # Median distance threshold
        self.left_tr = None  # Inner subtree
        self.right_tr = None  # Outer subtree
        self.points = None  # Leaf points
        self.centroid = None  # Center of the points (for leaves)
        self.radius = 0  # Max radius of the points (for leaves)

class VPTree:
    def __init__(self, leaf_size=50, use_max_variance=True):
        """
        VP Tree constructor.

        Parameters:
        - leaf_size (int): Maximum size of a leaf node.
        - use_max_variance (bool): If True, selects the vantage point using max variance of distances.
                                    If False, always selects the first point as the vantage point.
        """
        self.root = None
        self.leaf_size = leaf_size
        self.use_max_variance = use_max_variance

    #*-----------------------------------------------------------------------------
    def select_vantage_point(self, dataset, sample_size=200):
        """
        Selects a vantage point from the dataset by maximizing variance of distances.
        """
        num_points = dataset.shape[0]
        sample_size = min(sample_size, num_points)  # Adjust sample size dynamically
        sample_points = dataset[:sample_size]  # Take first `sample_size` points

        # Compute pairwise distances within the sample
        dist_matrix = np.linalg.norm(sample_points[:, None] - sample_points, axis=2)

        # Compute variance of distances
        mean_distances = np.mean(dist_matrix, axis=1)
        variance_distances = np.mean((dist_matrix - mean_distances[:, None])**2, axis=1)

        return np.argmax(variance_distances)  # Index of vantage point

    #*-----------------------------------------------------------------------------
    def build(self, points):
        """
        Build the VP-Tree from a set of points.
        """
        if points is None or len(points) == 0:
            return None

        points = np.asarray(points)  # Ensure points is a numpy array
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            # Stop if we reach leaf size
            if n_pts <= self.leaf_size:
                node.points = pts
                node.centroid = np.mean(pts, axis=0)  # Compute centroid
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))  # Compute max radius
                continue

            # Select vantage point
            if self.use_max_variance:
                sample_size = min(200, max(1, n_pts // 100))  # Ensure at least 1 sample
                vantage_idx = self.select_vantage_point(pts, sample_size=sample_size)
            else:
                vantage_idx = 0  # Always pick the first point

            node.point = pts[vantage_idx]
            remaining_pts = np.delete(pts, vantage_idx, axis=0)  # Remove vantage point

            if len(remaining_pts) == 0:
                # Handle the case where there's only one point
                node.points = np.array([node.point])
                node.centroid = node.point
                node.radius = 0
                continue

            # Compute distances from the vantage point
            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2

            # Create distance tuples for quickselect as (distance, index)
            # This matches the expected format of your quickselect function
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]

            # Use quickselect to find the median distance
            # This will return a tuple of (distance, index)
            median_tuple = quickselect(dist_tuples, median_idx)

            # Extract the median distance value from the tuple
            node.mu = median_tuple[0]

            # Partition into left (closer) and right (farther)
            left_mask = distances <= node.mu
            right_mask = ~left_mask
            left_pts = remaining_pts[left_mask]
            right_pts = remaining_pts[right_mask]

            # Create child nodes if they have points
            if len(left_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, left_pts))

            if len(right_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, right_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an exact nearest neighbor search in the VP Tree using triangle inequality.
        Optimized for practical performance based on empirical testing.

        Parameters:
            query (np.ndarray): The query point.

        Returns:
            tuple: (distance, nearest_point) - The closest point and its distance.
        """
        if self.root is None:
            return float('inf'), None

        query = np.asarray(query)
        best_dist = float("inf")
        best_point = None

        # Simple stack-based approach (empirically faster)
        stack = [self.root]

        while stack:
            node = stack.pop()  # Simple LIFO stack

            if node is None:
                continue

            # If this is a leaf node, check all stored points
            if node.points is not None:
                # Use ball-within-bounds test before checking individual points
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)

                    # Skip this node if it can't contain a closer point
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Check all points in this leaf
                dists = np.linalg.norm(node.points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node.points[min_idx]
                continue

            # For internal nodes, compute distance to the vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)

                # Update best nearest neighbor if vantage point is closer
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

                # Determine which subtree to explore first based on distance
                if dist_to_vp <= node.mu:
                    # Point is inside median radius, check inner subtree first
                    # We push the outer subtree first so inner gets popped first (LIFO)
                    if node.right_tr and abs(dist_to_vp - node.mu) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                else:
                    # Point is outside median radius, check outer subtree first
                    # We push the inner subtree first so outer gets popped first (LIFO)
                    if node.left_tr and abs(dist_to_vp - node.mu) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
            else:
                # Handle the case where node doesn't have a vantage point
                if node.right_tr:
                    stack.append(node.right_tr)
                if node.left_tr:
                    stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    @staticmethod
    def distance(p1, p2):
        """Optimized Euclidean distance"""
        diff = p1 - p2
        return np.sqrt(np.dot(diff, diff))

**-------------- Higgs ---------------------------------**

**First Point as VP (Run 1)**

In [ ]:
# Test code VP  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=False)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1302.61it/s]

Build Time: 25.20 sec, Search Time: 0.38 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 25.20 sec, Search: 0.38 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 25.20 sec, Total Search Time: 0.38 sec


**VP First Point (Run 2)**

In [ ]:
# Test code VP  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=False)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1117.68it/s]

Build Time: 24.54 sec, Search Time: 0.44 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 24.54 sec, Search: 0.44 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 24.54 sec, Total Search Time: 0.44 sec


**First Point as VP (Run 3)**

In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=False)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1307.72it/s]

Build Time: 24.56 sec, Search Time: 0.38 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 24.56 sec, Search: 0.38 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 24.56 sec, Total Search Time: 0.38 sec


**Max Variance as VP (Run 1)**

In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=True)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1268.62it/s]

Build Time: 28.14 sec, Search Time: 0.39 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 28.14 sec, Search: 0.39 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 28.14 sec, Total Search Time: 0.39 sec


**Max Var as VP (Run 2)**

In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=True)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1246.29it/s]

Build Time: 28.03 sec, Search Time: 0.40 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 28.03 sec, Search: 0.40 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 28.03 sec, Total Search Time: 0.40 sec


**Variance as VP (Run 3)**

In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=True)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1302.43it/s]

Build Time: 28.05 sec, Search Time: 0.38 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 28.05 sec, Search: 0.38 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 28.05 sec, Total Search Time: 0.38 sec


**Variance Check of Results**

**First PT as VP Data**

In [ ]:
import numpy as np

# Build and search times per experiment
build = np.array([24.56, 24.54, 25.20])
search = np.array([0.38, 0.44, 0.38])

# Compute total times
total_times = build + search

# Compute average total time
average_total_time = np.mean(total_times)

# Compute standard deviation for total times
total_std = np.std(total_times, ddof=0)  # Population standard deviation

# Display results
print("Average Total Time:", average_total_time)
print("Standard Deviation of Total Time:", total_std)

cv = (total_std / average_total_time) * 100
print("Coefficient of Variation (%):", cv)

Average Total Time: 25.166666666666668
Standard Deviation of Total Time: 0.29272664533466836
Coefficient of Variation (%): 1.1631522331178874


**Max Var as VP Data**

In [ ]:
# First Point as VP
build = np.array([28.05, 28.03, 28.14])
search = np.array([0.38, 0.40, 0.39])

# Compute total times
total_times = build + search

# Compute average total time
average_total_time = np.mean(total_times)

# Compute standard deviation for total times
total_std = np.std(total_times, ddof=0)  # Population standard deviation

# Display results
print("Average Total Time:", average_total_time)
print("Standard Deviation of Total Time:", total_std)

cv = (total_std / average_total_time) * 100
print("Coefficient of Variation (%):", cv)

Average Total Time: 28.463333333333335
Standard Deviation of Total Time: 0.04714045207910384
Coefficient of Variation (%): 0.16561817102390386


**-------------------------- Hepmass ------------------------------------**

In [ ]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 6 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [ ]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/hepmass_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [ ]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**First Point as VP (Run 1)**

In [ ]:
# Test code VP  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=False)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1321.24it/s]

Build Time: 25.42 sec, Search Time: 0.37 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 25.42 sec, Search: 0.37 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 25.42 sec, Total Search Time: 0.37 sec


**First Point as VP (Run 2)**

In [ ]:
# Test code VP  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=False)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1264.05it/s]

Build Time: 25.90 sec, Search Time: 0.39 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 25.90 sec, Search: 0.39 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 25.90 sec, Total Search Time: 0.39 sec


**First Point as VP (Run 3)**

In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=False)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1323.43it/s]

Build Time: 25.60 sec, Search Time: 0.37 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 25.60 sec, Search: 0.37 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 25.60 sec, Total Search Time: 0.37 sec


**Max Var as VP (Run 1)**

In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=True)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1290.29it/s]

Build Time: 28.01 sec, Search Time: 0.38 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 28.01 sec, Search: 0.38 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 28.01 sec, Total Search Time: 0.38 sec


**Max Var as VP (Run 2)**

In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=True)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1288.45it/s]

Build Time: 28.84 sec, Search Time: 0.38 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 28.84 sec, Search: 0.38 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 28.84 sec, Total Search Time: 0.38 sec


**Max Var as VP (Run 3)**

In [ ]:
# Test code KD  Tree
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
leaf_size = 50


for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the KD  tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, use_max_variance=True)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.search(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP Tree Search Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")


1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1317.17it/s]

Build Time: 28.38 sec, Search Time: 0.38 sec

=== Summary of VP Tree Search Tests ===
Data: 1000000 points, Queries:  500 => Build: 28.38 sec, Search: 0.38 sec
--------------------------------------------

=== Summary of VP Tree Search Tests ===

Total Build Time: 28.38 sec, Total Search Time: 0.38 sec


**Standard Deviation and CV**

**First Point as VP Data**

In [ ]:
import numpy as np

# Build and search times per experiment
build = np.array([25.42, 25.9, 25.6])
search = np.array([0.37, 0.39, 0.37])

# Compute total times
total_times = build + search

# Compute average total time
average_total_time = np.mean(total_times)

# Compute standard deviation for total times
total_std = np.std(total_times, ddof=0)  # Population standard deviation

# Display results
print("Average Total Time:", average_total_time)
print("Standard Deviation of Total Time:", total_std)

cv = (total_std / average_total_time) * 100
print("Coefficient of Variation (%):", cv)

Average Total Time: 26.016666666666666
Standard Deviation of Total Time: 0.2067741655902761
Coefficient of Variation (%): 0.7947757806160517


**Max Var as VP Data**

In [ ]:
import numpy as np

# Build and search times per experiment
build = np.array([28.01, 28.84, 28.38])
search = np.array([0.38, 0.38, 0.38])

# Compute total times
total_times = build + search

# Compute average total time
average_total_time = np.mean(total_times)

# Compute standard deviation for total times
total_std = np.std(total_times, ddof=0)  # Population standard deviation

# Display results
print("Average Total Time:", average_total_time)
print("Standard Deviation of Total Time:", total_std)

cv = (total_std / average_total_time) * 100
print("Coefficient of Variation (%):", cv)

Average Total Time: 28.790000000000003
Standard Deviation of Total Time: 0.3395094500402989
Coefficient of Variation (%): 1.179261722960399
